In [2]:
!pip install transformers
!pip install datasets
!pip install torch

In [4]:
from datasets import load_dataset
import numpy as np
import torch
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
)
from pprint import pformat
import gc
import random
import argparse
import logging
from datetime import datetime

In [5]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    # EarlyStoppingCallback,
)

In [9]:
def loader(dataset_name, tokenizer):
    assert dataset_name in dataset_cols
    sentence_col = dataset_cols[dataset_name]
    d_types = dataset_types.get(dataset_name, None)
    tot = []
    if d_types:
        for d_type in d_types:
            data = load_dataset(dataset_name, d_type, cache_dir="/scratch/amq259/datasets")
            tot.append(_preprocess_dataset(dataset_name, data, sentence_col, tokenizer))
    else:
        data = load_dataset(dataset_name, cache_dir="/scratch/amq259/datasets")
        tot.append(_preprocess_dataset(dataset_name, data, sentence_col, tokenizer))
    return tot

def _preprocess_dataset(dataset_name, data, sentence_col, tokenizer):
    preprocess_function = dataset_preprocess.get(dataset_name, lambda x: x)
    data = data.map(lambda x: {"input_text": preprocess_function(x[sentence_col])})
    data = data.map(
        lambda x: tokenizer(x["input_text"], padding="max_length", truncation=True),
        batched=True,
    )
    return data

In [6]:
eval_model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [22]:
trainer = Trainer(model=eval_model)

In [18]:
dataset_cols = {
    "EMBO/biolang": "special_tokens_mask",
    "empathetic_dialogues": "utterance",
    "conv_ai_3": "answer",
    "air_dialogue": "dialogue",
    "ted_talks_iwslt": "translation",
    "tweet_eval": "text",
}
dataset_types = {
    "ted_talks_iwslt": ["nl_en_2014", "nl_en_2015"],
    "tweet_eval": [
        "emoji",
        "emotion",
        "hate",
        "irony",
        "offensive",
        "sentiment",
        "stance_abortion",
        "stance_atheism",
        "stance_climate",
        "stance_feminist",
        "stance_hillary",
    ],
}
dataset_preprocess = {
    # 'NAME: <SENTENCE>'
    # "air_dialogue": lambda x: x.split("")[-1],
    "ted_talks_iwslt": lambda x: x["en"],
}

In [10]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased', use_fast=True)

In [7]:
EVALUATION_DATASET = 'ted_talks_iwslt'

In [8]:
datasets_to_eval = [EVALUATION_DATASET]

In [12]:
dataset_name = EVALUATION_DATASET

In [19]:
tot = loader(dataset_name, tokenizer)

Reusing dataset ted_talks_iwslt (/scratch/amq259/datasets/ted_talks_iwslt/nl_en_2014/1.1.0/43935b3fe470c753a023642e1f54b068c590847f9928bd3f2ec99f15702ad6a6)


Dataset ted_talks_iwslt downloaded and prepared to /scratch/amq259/datasets/ted_talks_iwslt/nl_en_2015/1.1.0/43935b3fe470c753a023642e1f54b068c590847f9928bd3f2ec99f15702ad6a6. Subsequent calls will reuse this data.


In [23]:
dataset_num = 0

for eval_dataset in tot:
    for split in eval_dataset:
        logging.info(f"Evaluating {eval_dataset[split]}")
        results = trainer.predict(eval_dataset[split])

        logging.info(f"Metrics: {results.metrics}")
        now = datetime.now()

        current_time = now.strftime("%H:%M:%S")
        filename = f"{EVALUATION_DATASET}-{dataset_num}-{split}-{current_time}-eval.out"
        np.savetxt(filename, results.predictions)
    
    dataset_num += 1